In [ ]:
%matplotlib inline
from collections import defaultdict
import torch, json, time, csv
import numpy as np
import graphviz
import pickle

att_filepath = '../../data/imagenet/attrann.mat'

# import scipy.io
# mat = scipy.io.loadmat(att_filepath)
# print('mat.keys()', mat.keys())
# # print('mat[__globals__]', mat['__globals__'])
# # print('mat[attrann].shape', mat['attrann'].shape)
# # print('mat[attrann]', mat['attrann'])
# # print('mat[attrann][0]', mat['attrann'][0])
# print('mat[attrann][0][0]', mat['attrann'][0][0])
# print('mat[attrann][0][0][0]', mat['attrann'][0][0][0])

def vec_to_str(v):
    K = v.size(0)
#     v = v.clamp(min=0)
    return ''.join([str(x) for x in v.tolist()])

def print_bin_mat(mat):
    rows, cols = mat.size()
    for r in range(rows):
        line = ''
        for c in range(cols):
            char = '*' if mat[r, c] == 0 else ' '
            line += char
        print(line)

import mat4py
mat = mat4py.loadmat(att_filepath)
print('mat[attrann.keys()', mat['attrann'].keys())
print('mat[attrann[images][:10]', mat['attrann']['images'][:10])
print('len(mat[attrann[images])', len(mat['attrann']['images']))
print('mat[attrann[attributes]', mat['attrann']['attributes'])
print('len(mat[attrann[attributes])', len(mat['attrann']['labels']))
print('len(mat[attrann[attributes][0])', len(mat['attrann']['labels'][0]))
att_t = torch.LongTensor(mat['attrann']['labels'])
att_t = att_t.clamp(min=0)
print('att_t.size()', att_t.size())
# print_bin_mat(att_t[:5])

N, K = att_t.size()
print('N', N, 'K', K)
# values = torch.LongTensor(N)
for n in range(5):
    print(n, vec_to_str(att_t[n]))

print('att_t.sum(dim=0)', att_t.sum(dim=0))    

values_count = defaultdict(int)
for n in range(N):
    v_str = vec_to_str(att_t[n])
    values_count[v_str] += 1

print('len(values_count)', len(values_count))
viable_sum = 0
l = []
for v, c in sorted(values_count.items()):
    if c >= 4:
        viable_sum += 1
    print(v, c)
    l.append((v, c))
print('')
l.sort(key=lambda x: x[1], reverse=True)
for v, c in l:
    print(v, c)
print('')
print('viable_sum', viable_sum)

# for k in range(K):
#     values = values * 2 + att_t[:, k]
# print('values[:10]', values[:10])

label_by_clsid = {}
label_by_idtxt = {}
with open('../../data/imagenet/map_clsloc.txt', 'r') as f:
    for line in f:
        line = line.strip()
        idtxt, clsid, label = line.split(' ')
        label_by_clsid[int(clsid)] = label
        label_by_idtxt[idtxt] = label
for i in range(5):
    print(i, label_by_clsid[i + 1])

def load_imagenet():
    with open('../../data/imagenet/val_data', 'rb') as f:
        d = pickle.load(f)
    print(d.keys())

    print(d['labels'][:5])

    img_size = 32

    x = d['data']
    y = d['labels']
    # mean_image = d['mean']

    x = x/np.float32(255)
    # mean_image = mean_image/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]
    data_size = x.shape[0]

    # x -= mean_image

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3)).transpose(0, 3, 1, 2)

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    print('X_train.shape', X_train.shape)
    return X_train, Y_train

X_train, Y_train = load_imagenet()

def plot_image(img):
    img = img.transpose(1,2,0)
    plt.imshow(img)

    
die()

nodes = set()
links = []
with open('../../data/imagenet/wordnet.is_a.txt', 'r') as f:
    for row in f:
        row = row.strip()
        p, n = row.split(' ')
        links.append((p, n))
        nodes.add(p)
        nodes.add(n)
print(links[:5])
# print(contents[:100])
print(list(nodes)[:5])

synset_id_to_name = {}
with open('../../data/imagenet/imagenet_label_to_wordnet_synset.txt', 'r') as f:
    lookup = eval(f.read())
#     lookup = json.loads(f.read().replace('\'', '"'))
    for _, d in list(lookup.items()):
        id, n = d['id'].split('-')
        id = n + id
        synset_id_to_name[id] = d['label']
#         print(d['id'], d['label'])

print(list(synset_id_to_name.items())[:5])

dot = graphviz.Digraph('foobar')
# for i in range(20):
for node in list(nodes)[:10]:
    dot.node(synset_id_to_name[node])

#     if i > 0:
#         dot.edge(f'a{i-1}', 'a{i}')
dot.render()
        
# {0: {'id': '01440764-n',
#      'label': 'tench, Tinca tinca',
#      'uri': 'http://wordnet-rdf.princeton.edu/wn30/01440764-n'},
#  1: {'id': '01443537-n',
#      'label': 'goldfish, Carassius auratus',
#      'uri': 'http://wordnet-rdf.princeton.edu/wn30/01443537-n'},
#  2: {'id': '01484850-n',
    